In [ ]:
!tar zxvf ./drive/My\ Drive/Colab\ Notebooks/train.tgz
!pip install wandb

In [ ]:
!wandb login

# CAPTCHA Recognition - 6 digits

In [1]:
import tensorflow as tf
import densenet

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


# Parsing Data

In [ ]:
CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

#functions
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
        #crop to [60,30,3]
    img = tf.io.decode_jpeg(a)
    return img

In [ ]:
#training data 01 
train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data01_train/"
data = pd.read_csv("./train/data01_train.csv")
for i in range(len(data)):
    data.iloc[i,1] = list(data.iloc[i,1])
arr = np.zeros([6,50000],str)
for j in range(6):
    for i in range(len(data)):
        arr[j][i] = data.iloc[i,1][j]
data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

for i in range(6):
    data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
train_data = train_dir.map(lambda x: readimg_to_tensor(x))
train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
train_data01=tf.data.Dataset.zip((train_data,train_label))

In [ ]:
#training data 02
train_dir_data02 = os.path.abspath(os.getcwd())+"/train/data02_train/"
data2 = pd.read_csv("./train/data02_train.csv")
for i in range(len(data2)):
    data2.iloc[i,1] = list(data2.iloc[i,1])
arr = np.zeros([6,50000],str)
for j in range(6):
    for i in range(len(data2)):
        arr[j][i] = data2.iloc[i,1][j]
data2 = data2.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

#to one hot
for i in range(6):
    data2.iloc[:,i+1] = data2.iloc[:,i+1].map(parse_label)

TL = tf.convert_to_tensor(list(data2.iloc[:,1].values))
TL2 = tf.convert_to_tensor(list(data2.iloc[:,2].values))
TL3 = tf.convert_to_tensor(list(data2.iloc[:,3].values))
TL4 = tf.convert_to_tensor(list(data2.iloc[:,4].values))
TL5 = tf.convert_to_tensor(list(data2.iloc[:,5].values))
TL6 = tf.convert_to_tensor(list(data2.iloc[:,6].values))

train_dir2 = tf.data.Dataset.list_files(train_dir_data02+'*.jpg',shuffle=False) 
train_data2 = train_dir2.map(lambda x: readimg_to_tensor(x))
train_label2 = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
train_data02=tf.data.Dataset.zip((train_data2,train_label2))

In [ ]:
#number of element
num_elements = tf.data.experimental.cardinality(train_data01).numpy()
num_elements

# Model configeration

In [ ]:
import wandb

In [33]:
model = densenet.DenseNet(input_shape=(224, 224, 3),
             nb_dense_block=4,
             growth_rate=32,
             nb_filter=64,
             nb_layers_per_block=[6,6,24,12],
             bottleneck=False,
             reduction=0.5,
             dropout_rate=0.0,
             weight_decay=1e-4,
             subsample_initial_block=True,
             include_top=True,
             weights=None,
             input_tensor=None,
             pooling=None,
             classes=10,
             activation='softmax',
             transition_pooling='avg')

In [34]:
model.summary()

0][0]              
__________________________________________________________________________________________________
dense_2_15_conv2D (Conv2D)      (None, 14, 14, 32)   184320      activation_465[0][0]             
__________________________________________________________________________________________________
concatenate_382 (Concatenate)   (None, 14, 14, 672)  0           concatenate_381[0][0]            
                                                                 dense_2_15_conv2D[0][0]          
__________________________________________________________________________________________________
dense_2_16_bn (BatchNormalizati (None, 14, 14, 672)  2688        concatenate_382[0][0]            
__________________________________________________________________________________________________
activation_466 (Activation)     (None, 14, 14, 672)  0           dense_2_16_bn[0][0]              
_________________________________________________________________________________________

In [5]:
model2 = tf.keras.applications.DenseNet121()

33193984/33188688 [==============================] - 8s 0us/step


In [6]:
model2.summary()

_bn[0][0]          
__________________________________________________________________________________________________
conv5_block4_1_conv (Conv2D)    (None, 7, 7, 128)    77824       conv5_block4_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_block4_1_bn (BatchNormali (None, 7, 7, 128)    512         conv5_block4_1_conv[0][0]        
__________________________________________________________________________________________________
conv5_block4_1_relu (Activation (None, 7, 7, 128)    0           conv5_block4_1_bn[0][0]          
__________________________________________________________________________________________________
conv5_block4_2_conv (Conv2D)    (None, 7, 7, 32)     36864       conv5_block4_1_relu[0][0]        
__________________________________________________________________________________________________
conv5_block4_concat (Concatenat (None, 7, 7, 640)    0           conv5_block3_concat[0][0